In [ ]:
!pip install langchain-community
!pip install pypdf
!pip install requests
!pip install beautifulsoup4


In [29]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# Charge tous les PDF du dossier avec PyPDFLoader
loader = DirectoryLoader(
    path="rag_master",                   # dossier contenant les PDF
    glob="**/*.pdf",                       # extension visée
    loader_cls=PyPDFLoader                 # type de loader utilisé pour chaque fichier
)
pdf_documents = loader.load()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)


In [30]:
print(pdf_documents[0].page_content.splitlines()[0])

Filières NG de la FSDM dans le cadre du PACTE ESRI 2030          Offre de formation 2024-2025 Masters  Masters Brochures BEVP : Biotechnologie, Ecologie et Valorisation des Phytoressources Affiche BAMQ (filière d'excellence) : Biotechnologie Alimentaire et Management de la Qualité Affiche PNOMER : Physique des nouveaux matériaux et énergies renouvelables  Affiche M2A : Matériaux Avancés et Applications Affiche MAER : Automatique et Énergies Renouvelables  Affiche 2ESI : Électronique Embarquée Et Systèmes Intelligents  Affiche M2SI : Smart Industry  Affiche CAE : Chimie Analytique et Environnement Affiche CMI : Chimie des Matériaux Innovants  Affiche MASI : Mathématiques Appliquées et Systèmes Intelligents  Affiche MMP : Mathématiques pures  Affiche MSSD : Modélisation Stochastique et Statistique Décisionnelle  Affiche MQL : Qualité du Logiciel  Affiche BDSI : Big Data et Systèmes intelligents Affiche WISD : Web Intelligence et Sciences des Données  Affiche MLAIM : Machine Learning Avan

In [31]:
print(f"{len(pdf_documents)} documents chargés.")

21 documents chargés.


In [ ]:
import re
import unicodedata
import string

def nettoyage_avance(texte):
    # 1. Normaliser les caractères Unicode
    texte = unicodedata.normalize('NFKC', texte)

    # 2. Supprimer les sauts de lignes multiples et remplacer par un espace
    texte = re.sub(r'\n+', ' ', texte)

    # 3. Supprimer les césures ("mot-\nsuivant" → "motsuivant")
    texte = re.sub(r'-\s+', '', texte)

    # 4. Supprimer les numéros de page
    texte = re.sub(r'(Page\s*\d+)', '', texte, flags=re.IGNORECASE)
    texte = re.sub(r'^\s*\d+\s*$', '', texte, flags=re.MULTILINE)

    # 5. Supprimer tabulations et espaces multiples
    texte = re.sub(r'[\t\r\f\v]+', ' ', texte)
    texte = re.sub(r' +', ' ', texte)

    # 6. Supprimer la ponctuation (virgules, points, etc.)
    texte = texte.translate(str.maketrans('', '', string.punctuation))

    # 7. Supprimer espaces début/fin
    texte = texte.strip()

    return texte


Filières NG de la FSDM dans le cadre du PACTE ESRI 2030 Offre de formation 20242025 Masters Masters Brochures BEVP Biotechnologie Ecologie et Valorisation des Phytoressources Affiche BAMQ filière dexcellence Biotechnologie Alimentaire et Management de la Qualité Affiche PNOMER Physique des nouveaux matériaux et énergies renouvelables Affiche M2A Matériaux Avancés et Applications Affiche MAER Automatique et Énergies Renouvelables Affiche 2ESI Électronique Embarquée Et Systèmes Intelligents Affiche M2SI Smart Industry Affiche CAE Chimie Analytique et Environnement Affiche CMI Chimie des Matériaux Innovants Affiche MASI Mathématiques Appliquées et Systèmes Intelligents Affiche MMP Mathématiques pures Affiche MSSD Modélisation Stochastique et Statistique Décisionnelle Affiche MQL Qualité du Logiciel Affiche BDSI Big Data et Systèmes intelligents Affiche WISD Web Intelligence et Sciences des Données Affiche MLAIM Machine Learning Avancé et Intelligence Multimédia Affiche
21 documents nettoy

In [32]:
def extract_text_from_url(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, "html.parser")
    for tag in soup(["script", "style", "header", "footer", "nav", "form"]):
        tag.decompose()
    text = soup.get_text(separator='\n')
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    return "\n".join(lines)

urls = {
    "masters": "https://www.fsdm.usmba.ac.ma/Formations/Master",
    "candidature": "https://www.fsdm.usmba.ac.ma/Acces/Candidature2025"
}

In [33]:
scraped_documents = []
for name, url in urls.items():
    raw_text = extract_text_from_url(url)
    scraped_documents.append(Document(page_content=raw_text, metadata={"source": url}))

In [35]:
print(pdf_documents[0].page_content.splitlines()[0])

Filières NG de la FSDM dans le cadre du PACTE ESRI 2030 Offre de formation 20242025 Masters Masters Brochures BEVP  Biotechnologie Ecologie et Valorisation des Phytoressources Affiche BAMQ filière dexcellence  Biotechnologie Alimentaire et Management de la Qualité Affiche PNOMER  Physique des nouveaux matériaux et énergies renouvelables Affiche M2A  Matériaux Avancés et Applications Affiche MAER  Automatique et Énergies Renouvelables Affiche 2ESI  Électronique Embarquée Et Systèmes Intelligents Affiche M2SI  Smart Industry Affiche CAE  Chimie Analytique et Environnement Affiche CMI  Chimie des Matériaux Innovants Affiche MASI  Mathématiques Appliquées et Systèmes Intelligents Affiche MMP  Mathématiques pures Affiche MSSD  Modélisation Stochastique et Statistique Décisionnelle Affiche MQL  Qualité du Logiciel Affiche BDSI  Big Data et Systèmes intelligents Affiche WISD  Web Intelligence et Sciences des Données Affiche MLAIM  Machine Learning Avancé et Intelligence Multimédia Affiche


In [36]:
print(scraped_documents[0].page_content.splitlines()[0])

Faculté des Sciences Fès Masters Masters Recheche  Spécialisé MASTERS Master Coordonnateur Département Affiche Présentation Matériaux et Applications pour l’Energie Photovoltaïque MAEP Pr SSAYOURI Physique Physique des Nouveaux Matériaux et Energies Renouvelables PNOMER Pr AOUAZZANI TAIBI Physique Master Chimie MC Pr ZRAIS Chimie Ecologie et Préservation de l’Environnement pour un Développement Durable EP2ED Pr RGUEMMOUH Biologie Sciences Biologiques et Santé SANBIOL Pr BLYOUSSI Biologie Biotechnologie Ecologie et Valorisation des Phytoressources BEVP Pr JBAHHOU Biologie Mathématiques pures MMP Pr ATAJMOUATI Mathématiques Mathématiques Appliquées et Science des données MASD Pr JBENNOUNA Mathématiques Informatique Décisionnelle et Vision Intelligente MIDVI Pr JBOUMHIDI Informatique Systèmes Intelligents et Décisionnels MSID Pr NENNAHNAHI Informatique Web Intelligence et Sciences des Données WISD Pr AYAHYAOUY Informatique BIG DATA ANALYTICS  SMART SYSTEMS BDSaS Pr AZINEDINE Informatique 

In [34]:
# 3. Concaténer les listes
all_documents = pdf_documents + scraped_documents

# 4. Appliquer le nettoyage sur tous les documents
for doc in all_documents:
    doc.page_content = nettoyage_avance(doc.page_content)
